In [1]:
%pylab
%matplotlib inline
%load_ext watermark
%watermark -v -p numpy,tensorflow

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib
CPython 3.5.6
IPython 6.5.0

numpy 1.15.2
tensorflow 1.11.0


In [2]:
import tensorflow as tf

텐서플로 1.7.0 버전에서부터는 샘플 데이터를 다운로드하는 기능이 제외될 예정이라는 경고가 발생합니다. 대신 케라스(Keras)를 사용하여 MNIST 데이터를 다운받습니다.

In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

배치 데이터를 만들기 위해 파이썬 제너레이터 함수를 정의합니다.

In [4]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [5]:
tf.convert_to_tensor(X_train).get_shape()

TensorShape([Dimension(60000), Dimension(784)])

가중치 텐서와 바이어스 텐서를 만듭니다.

In [6]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

훈련 이미지 데이터를 넣을 플레이스홀더와 소프트맥스 텐서를 만듭니다.

In [7]:
x = tf.placeholder("float", [None, 784])
y = tf.nn.softmax(tf.matmul(x,W) + b)

실제 레이블을 담기위한 텐서와 교차 엔트로피 방식을 이용하는 그래디언트 디센트 방식을 선택합니다.

In [8]:
y_ = tf.placeholder("float", [None,10])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

변수를 초기화하고 세션을 시작합니다.

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

1000의 반복을 수행하고 결과를 출력합니다. 최종 정확도는 91% 정도 입니다.

In [10]:
for i in range(1000):
    batch_xs, batch_ys = next(shuffle_batch(X_train, y_train, 100))
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    if i % 100 == 0:
        print(sess.run(accuracy, feed_dict={x: X_test, y_: y_test}))

0.2089
0.8446
0.9044
0.9046
0.9089
0.9138
0.9154
0.9148
0.909
0.915
